# Install

# Import

In [1]:
import sys
import os
import gc
import copy
import yaml
import pickle
import random
import joblib 
import shutil
from time import time
import typing as tp
from pathlib import Path
import psutil

import numpy as np
import pandas as pd
import scipy

from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import average_precision_score as APS
import duckdb


import torch
import torchvision
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.cuda import amp
from torch.nn import BCELoss
from torch.utils.data import Dataset


import timm
from mamba_ssm import Mamba
from transformers import AutoModel, AutoTokenizer

import albumentations as A
from albumentations.pytorch import ToTensorV2


# use one device only
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
con = duckdb.connect()

20000_50per_CLM.parquet

In [2]:
class CFG:
    TRAIN_CLM_PATH = Path('../../data/processed/20000_50per_CLM.parquet')
    TEST_ENC_PATH = Path('../../data/external/test_enc.parquet')
    TRAIN_PATH = Path('../../data/raw/train.parquet')
    TEST_PATH = Path('../../data/raw/test.parquet')
    folds = 2
    max_epoch = 9             # number of max epoch. 1epoch means going around the training dataset.
    batch_size = 8           # batch size. Number of samples passed to the network in one training step
    lr = 1.0e-03              # learning rate. determine step size when updating model's weight
    weight_decay = 1.0e-02    # weight decay. Append regularization term for prevent over fitting
    es_patience = 5           # Timing for early stopping. If there is no improvement within this number of epochs, training will be stopped early.
    seed = 1086               # Random number seed
    deterministic = True      # Enable/disable deterministic behavior. If enabled, the program will produce the same results every time it starts with the same initial conditions and inputs.
    enable_amp = False        # Enable/disable Automatic Mixed Precision. Optimizations for floating point etc.
    device = "cuda" 
    n_classes = 3

In [3]:
# train = pd.read_parquet(CFG.TRAIN_ENC_PATH)
# test = pd.read_parquet(CFG.TEST_ENC_PATH)
# train.head()
# print(len(train))
# print(len(test))

In [4]:
# display(train.head())
# print(len(train))
# print(len(test))

In [5]:
# train = con.query(f"""(SELECT *
#                         FROM parquet_scan('{CFG.TRAIN_PATH}')
#                         LIMIT 60000)""").df()
# test = con.query(f"""(SELECT *
#                         FROM parquet_scan('{CFG.TRAIN_ENC_PATH}')
#                         LIMIT 1674896)""").df()

In [6]:
train_clm = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TRAIN_CLM_PATH}')
                        )""").df()

In [7]:
test = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TEST_PATH}')
                        LIMIT 10000)""").df()

In [8]:
from sklearn.model_selection import KFold

def split_fold(df:pd.DataFrame):
    # config
    N_FOLDS = 5
    RANDAM_SEED = 42
    df['fold'] = -1

    # object
    skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDAM_SEED)

    for i, (train_index, test_index) in enumerate(skf.split(df)):
        df.loc[test_index, 'fold'] = i
    
    return df

train_clm = split_fold(train_clm)


In [9]:
train_clm.head()

,id,0,1,2,3,4,5,6,7,8,...,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
0,46212804,0.298898,0.215735,0.239334,0.034285,0.182660,0.425175,-0.200705,0.449326,-0.123180,...,-0.215490,-0.176117,0.369509,-0.111854,0,BRD4,0,0,0,0
1,61664237,0.382253,-0.172434,-0.062786,0.120820,-0.093798,0.520192,0.141752,0.332013,-0.035972,...,0.003045,-0.133553,0.263715,-0.389847,0,sEH,0,0,0,3
2,292563699,0.065111,-0.260247,0.179049,0.162758,0.168346,0.483988,-0.087336,0.153319,0.034525,...,-0.034983,-0.334333,0.258336,-0.173001,0,BRD4,0,0,0,4
3,68335434,0.133682,-0.071265,0.217721,0.085580,-0.079135,0.571667,0.020465,0.022752,0.190998,...,-0.151728,0.038851,0.198905,-0.346823,0,BRD4,0,0,0,0
4,142776429,-0.171107,0.095327,0.110521,0.179870,0.173595,0.240704,-0.268263,-0.235531,0.033824,...,0.061882,-0.400442,0.151702,-0.257879,0,BRD4,0,0,0,3


In [10]:
display(train_clm.head())
display(train_clm.tail())

,id,0,1,2,3,4,5,6,7,8,...,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
0,46212804,0.298898,0.215735,0.239334,0.034285,0.182660,0.425175,-0.200705,0.449326,-0.123180,...,-0.215490,-0.176117,0.369509,-0.111854,0,BRD4,0,0,0,0
1,61664237,0.382253,-0.172434,-0.062786,0.120820,-0.093798,0.520192,0.141752,0.332013,-0.035972,...,0.003045,-0.133553,0.263715,-0.389847,0,sEH,0,0,0,3
2,292563699,0.065111,-0.260247,0.179049,0.162758,0.168346,0.483988,-0.087336,0.153319,0.034525,...,-0.034983,-0.334333,0.258336,-0.173001,0,BRD4,0,0,0,4
3,68335434,0.133682,-0.071265,0.217721,0.085580,-0.079135,0.571667,0.020465,0.022752,0.190998,...,-0.151728,0.038851,0.198905,-0.346823,0,BRD4,0,0,0,0
4,142776429,-0.171107,0.095327,0.110521,0.179870,0.173595,0.240704,-0.268263,-0.235531,0.033824,...,0.061882,-0.400442,0.151702,-0.257879,0,BRD4,0,0,0,3


,id,0,1,2,3,4,5,6,7,8,...,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
19995,81985115,0.267760,-0.205158,0.055440,0.164821,-0.094532,0.485431,0.157744,0.006200,0.010068,...,-0.089555,-0.248966,0.285321,-0.347933,1,sEH,0,0,1,1
19996,227438280,0.325959,-0.188691,0.099676,0.252709,-0.037638,0.579446,-0.013899,0.089516,0.085101,...,-0.062313,-0.155229,0.309876,-0.362709,1,BRD4,1,0,0,2
19997,62009222,0.337114,-0.080732,0.233315,0.319879,0.102618,0.351365,-0.211420,0.025342,-0.010430,...,-0.387911,-0.392963,0.381071,-0.261988,1,sEH,0,0,1,4
19998,162837859,0.350395,-0.336577,-0.020381,0.266737,0.073077,0.627532,0.134965,0.158673,-0.056279,...,-0.095635,-0.438893,0.192547,-0.307759,1,HSA,0,1,0,0
19999,36449071,0.375762,-0.040645,0.166833,0.137244,0.009307,0.530286,0.003918,0.089983,0.129359,...,-0.074886,-0.066868,0.113061,-0.320741,1,HSA,0,1,0,4


# Dataset

In [11]:
class EXDataset(Dataset):
    def __init__(
        self,
        train: pd.DataFrame,
        label: pd.DataFrame = pd.DataFrame(),
        is_test: bool = False,
        transform = None
    ):
        self.train = train
        self.label = label
        self.is_test = is_test
        self.transform = transform
        
    def __len__(self):
        # return total num of data
        return len(self.train)
    
    def __getitem__(self, index:int):
        # return data and target assosiated with index
        X = self.train.iloc[index]
        X = self._apply_transform(X)
        
        if self.is_test:
            y = np.argmax(np.zeros(CFG.n_classes))
            # y = [0, 0, 0]
        else:
            y = np.argmax(self.label.iloc[index].values)
            # y = self.label.iloc[index].values
        return X, y
    
    def _apply_transform(self, X):
        if self.transform:
            X = self.transform(X)
        return X

# Model

In [12]:
class MambaModel(nn.Module):
    def __init__(self, 
                 dim_model=384, # Model dimension d_model (embedding size)
                 d_state=16, # SSM state expansion factor
                 d_conv=4, # Local convolution width
                 expand=2, # Block expansion factor
                 output = 3 # number of classes (or output number simply)
                 ):
        super().__init__()
        self.model = Mamba(
            d_model=dim_model,  
            d_state=d_state,  
            d_conv=d_conv,    
            expand=expand,    
        ).to("cuda")
        # mamba pass trought input size as is.
        self.output = nn.Linear(dim_model, output)
        self.softmax = nn.Softmax(dim=-1)
        

    def forward(self, x):
        # Add the length dimension if input has only 2 dimensions
        if len(x.shape) == 2:
            x = x.unsqueeze(1)
            
        x = self.model(x)
        x = self.output(x)
        x = x.squeeze()
        # x = self.softmax(x)
        return x

##### set seeds

In [13]:
def set_random_seed(seed: int = 42, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = deterministic  # type: ignore

##### move tensors to device

In [14]:
def to_device(
    tensors: tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]],
    device: torch.device, *args, **kwargs
):
    if isinstance(tensors, tuple):
        return (t.to(device, *args, **kwargs) for t in tensors)
    elif isinstance(tensors, dict):
        return {
            k: t.to(device, *args, **kwargs) for k, t in tensors.items()}
    else:
        return tensors.to(device, *args, **kwargs)

##### transform values to tensor

In [15]:
def to_tensor(x):
    return torch.tensor(x.values, dtype=torch.float32)

# Train

In [16]:
def train_one_fold(CFG,
                   val_fold: int,
                   train: pd.DataFrame,
                   output_path
                   ):
    feature_columns = [str(i) for i in range(384)]
    label_columns = ['bind1', 'bind2', 'bind3']

    set_random_seed(CFG.seed, deterministic=CFG.deterministic)
    device = torch.device(CFG.device)
    train_dataset = EXDataset(train = train[feature_columns][train['fold']!=val_fold].reset_index(drop=True), 
                              label = train[label_columns][train['fold']!=val_fold].reset_index(drop=True), 
                              transform = to_tensor)
    val_dataset = EXDataset(train = train[feature_columns][train['fold']==val_fold].reset_index(drop=True), 
                            label = train[label_columns][train['fold']==val_fold].reset_index(drop=True), 
                            transform = to_tensor)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)

    model = MambaModel()
    model.to(device)

    optimizer = optim.AdamW(params=model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = lr_scheduler.OneCycleLR(
        optimizer=optimizer, epochs=CFG.max_epoch,
        pct_start=0.0, steps_per_epoch=len(train_loader),
        max_lr=CFG.lr, div_factor=25, final_div_factor=4.0e-01
    )
    loss_func = nn.CrossEntropyLoss()
    loss_func.to(device)
    loss_func_val = nn.CrossEntropyLoss()

    use_amp = CFG.enable_amp
    scaler = amp.GradScaler(enabled=use_amp)

    best_val_loss = 1.0e+09
    best_epoch = 0
    train_loss = 0
    val_loss = 0
    
    for epoch in range(1, CFG.max_epoch + 1):
        epoch_start = time()
        model.train()
        for batch in train_loader:
            
            x, t = batch
            # print(x)
            # print(t)
            x = to_device(x, device)
            t = to_device(t, device).long()
            # sys.exit()
                
            optimizer.zero_grad()
            with amp.autocast(use_amp):
                y = model(x)
                loss = loss_func(y, t)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item()
            scheduler.step()
            
        train_loss /= len(train_loader)
            
        model.eval()
        for batch in val_loader:
            x, t = batch
            x = to_device(x, device)
            with torch.no_grad(), amp.autocast(use_amp):
                y = model(x)
#                 y = torch.sigmoid(y)
            y = y.detach().cpu().to(torch.float32)
            loss = loss_func_val(y, t)
            val_loss += loss.item()
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_epoch = epoch
            best_val_loss = val_loss
            # print("save model")
            torch.save(model.state_dict(), str(output_path / f'snapshot_epoch_{epoch}.pth'))
        
        elapsed_time = time() - epoch_start
        print(
            f"[epoch {epoch}] train loss: {train_loss: .6f}, val loss: {val_loss: .6f}, elapsed_time: {elapsed_time: .3f}")
        
        if epoch - best_epoch > CFG.es_patience:
            print("Early Stopping!")
            break
            
        train_loss = 0
        val_loss = 0
            
    return val_fold, best_epoch, best_val_loss

Do the above function actually, and save the best model of each epoch.



In [17]:
score_list = []
for fold_id in range(CFG.folds):
    output_path = Path(f"fold{fold_id}")
    output_path.mkdir(exist_ok=True)
    print(f"[fold{fold_id}]")
    score_list.append(train_one_fold(CFG, fold_id, train_clm, output_path))

[fold0]
[epoch 1] train loss:  0.710691, val loss:  0.640049, elapsed_time:  7.282
[epoch 2] train loss:  0.622336, val loss:  0.617710, elapsed_time:  7.291
[epoch 3] train loss:  0.588847, val loss:  0.614598, elapsed_time:  7.223
[epoch 4] train loss:  0.561349, val loss:  0.586743, elapsed_time:  6.889
[epoch 5] train loss:  0.533353, val loss:  0.559209, elapsed_time:  6.925
[epoch 6] train loss:  0.506122, val loss:  0.549825, elapsed_time:  6.675
[epoch 7] train loss:  0.482496, val loss:  0.551723, elapsed_time:  7.209
[epoch 8] train loss:  0.459351, val loss:  0.549252, elapsed_time:  6.870
[epoch 9] train loss:  0.444053, val loss:  0.547983, elapsed_time:  7.372
[fold1]
[epoch 1] train loss:  0.715195, val loss:  0.640740, elapsed_time:  6.689
[epoch 2] train loss:  0.621290, val loss:  0.623129, elapsed_time:  7.091
[epoch 3] train loss:  0.588694, val loss:  0.598158, elapsed_time:  7.105
[epoch 4] train loss:  0.557465, val loss:  0.591780, elapsed_time:  7.316
[epoch 5]

Check the result.

In [18]:
print(score_list)

[(0, 9, 0.547982767470181), (1, 9, 0.5449692402929067)]


Delete models without best:

In [19]:
# select the best model and delete others
best_log_list = []
for (fold_id, best_epoch, _) in score_list:
    
    # select the best model
    exp_dir_path = Path(f"fold{fold_id}")
    best_model_path = exp_dir_path / f"snapshot_epoch_{best_epoch}.pth"
    # copy to new place
    copy_to = f"./best_model_fold{fold_id}.pth"
    shutil.copy(best_model_path, copy_to)
    
    for p in exp_dir_path.glob("*.pth"):
        # delete
        p.unlink()

# Infer

In [20]:
def run_inference_loop(model, loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(loader):
            x = to_device(batch[0], device)
            y = model(x)
            pred_list.append(y.detach().cpu().numpy())
    
    # concatenate to vertical (to df that like long scroll)
    pred_arr = np.concatenate(pred_list)
    del pred_list
    return pred_arr

Do inference actually.

In [21]:
def inference(train):
    test_pred_arr = np.zeros((len(train), CFG.n_classes))
    score_list = []

    for fold_id in range(CFG.folds):
        print(f"\n[fold {fold_id}]")
        device = torch.device(CFG.device)
        
        feature_columns = [str(i) for i in range(384)]
        test_dataset = EXDataset(test[feature_columns], transform = torch.tensor())
    
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)

        # get model
        model_path = f"./best_model_fold{fold_id}.pth"
        model = MambaModel()
        model.load_state_dict(torch.load(model_path, map_location=device))

        # inference
        test_pred = run_inference_loop(model, test_loader, device)
        test_pred_arr[fold_id] = test_pred

        del val_idx, val_path_label
        del model, val_loader
        torch.cuda.empty_cache()
        gc.collect()
    return test_pred_arr

test_preds_arr = inference(train_clm)


[fold 0]


KeyError: "None of [Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',\n       ...\n       '374', '375', '376', '377', '378', '379', '380', '381', '382', '383'],\n      dtype='object', length=384)] are in the [columns]"

mean each fold's predict.

In [ ]:
test_pred = test_preds_arr.mean(axis=0)

# Submission

In [ ]:
# soon...